## Scratchpad to use for Dive into Deep Learning (D2L) book
Using this as scratchpad to teach

In [65]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
# Use apple silicon mps
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [3]:
device = mps_device
print(device)

mps


## Torch autograd - L2

In [7]:
a = torch.tensor([2., 3], requires_grad=True)
b = torch.tensor([6., 4], requires_grad=True)

In [8]:
Q = 3*a**3 - b**2

In [9]:
external_grad = torch.tensor([1., 1])
Q.backward(gradient=external_grad)

In [10]:
print(a.grad, b.grad)

tensor([36., 81.]) tensor([-12.,  -8.])


In [19]:
a.grad.zero_()
b.grad.zero_()

tensor([0., 0.])

In [23]:
Q = 3*a**3 - b**2
external_grad = torch.tensor([1., 0])
Q.backward(gradient=external_grad) # J.T * v

In [24]:
print(a.grad, b.grad)

tensor([72.,  0.]) tensor([-24.,   0.])


## Building blocks - L3

In [30]:
from torch.utils import data

def synthesize_data(w, b, num_examples):
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = X @ w + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2., -3.4])
true_b = 4.2
features, labels = synthesize_data(true_w, true_b, 1000)
features.shape, labels.shape

(torch.Size([1000, 2]), torch.Size([1000, 1]))

In [33]:
def load_data(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [34]:
batch_size = 10
data_iter = load_data((features, labels), batch_size)

In [42]:
from torch import nn

net = nn.Sequential(
    nn.Linear(2, 1)
)
net[0].weight, net[0].bias

(Parameter containing:
 tensor([[-0.2316, -0.6634]], requires_grad=True),
 Parameter containing:
 tensor([0.6364], requires_grad=True))

In [50]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)
net[0].weight, net[0].bias

(Parameter containing:
 tensor([[ 0.0159, -0.0125]], requires_grad=True),
 Parameter containing:
 tensor([0.], requires_grad=True))

In [55]:
loss = nn.MSELoss()
optim = torch.optim.SGD(net.parameters(), lr=0.02)

In [56]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        optim.zero_grad()
        
        l = loss(net(X), y)
        l.backward()
        optim.step()
    l = loss(net(features), labels)
    print(f'epoch -> {epoch + 1} with loss -> {l:f}')
        

epoch -> 1 with loss -> 0.000100
epoch -> 2 with loss -> 0.000100
epoch -> 3 with loss -> 0.000100


In [63]:
w = net[0].weight.data
print(abs(true_w - w.reshape(true_w.shape)))

tensor([0.0013, 0.0002])


In [64]:
b = net[0].bias.data
print(abs(true_b - b))

tensor([0.0004])


## L4

In [67]:
net = nn.Sequential(
    nn.Linear(2, 4),
    nn.Linear(4, 1)
)
X = torch.rand(3, 2)
net(X)

tensor([[-0.5587],
        [-0.5892],
        [-0.7648]], grad_fn=<AddmmBackward0>)

In [68]:
net[0].state_dict()

OrderedDict([('weight',
              tensor([[-0.3745,  0.0698],
                      [-0.4380, -0.3627],
                      [-0.2849, -0.4553],
                      [ 0.6470, -0.2840]])),
             ('bias', tensor([-0.3878, -0.2440,  0.1550,  0.5828]))])

In [69]:
net[1].state_dict()

OrderedDict([('weight', tensor([[ 0.4145,  0.4723, -0.0189,  0.0957]])),
             ('bias', tensor([-0.1451]))])

In [73]:
for name, param in net.named_parameters():
    print(name, param)

0.weight Parameter containing:
tensor([[-0.3745,  0.0698],
        [-0.4380, -0.3627],
        [-0.2849, -0.4553],
        [ 0.6470, -0.2840]], requires_grad=True)
0.bias Parameter containing:
tensor([-0.3878, -0.2440,  0.1550,  0.5828], requires_grad=True)
1.weight Parameter containing:
tensor([[ 0.4145,  0.4723, -0.0189,  0.0957]], requires_grad=True)
1.bias Parameter containing:
tensor([-0.1451], requires_grad=True)


In [74]:
def blk1():
    return nn.Sequential(
        nn.Linear(4, 8),
        nn.ReLU(),
        nn.Linear(8, 4),
        nn.ReLU()
    )
    
def blk2():
    net = nn.Sequential()
    for i in range(2):
        net.add_module(f'blk_{i}', blk1())
    return net

In [76]:
blk_net = nn.Sequential(
    blk2(),
    nn.Linear(4, 1)
)
print(blk_net)

Sequential(
  (0): Sequential(
    (blk_0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (blk_1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


## L5

In [83]:
def init_weights_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

print(net)
net.apply(init_weights_normal)
net[0].weight.data[0], net[0].bias.data[0]

Sequential(
  (0): Linear(in_features=2, out_features=4, bias=True)
  (1): Linear(in_features=4, out_features=1, bias=True)
)


(tensor([-0.0043, -0.0085]), tensor(0.))

### Load and save params

```
torch.save(net.state_dict(), 'name')
```
```
clone = Net()
clone.load_state_dict(torch.load('name'))
```

## L6